<a href="https://colab.research.google.com/github/JLDC/hugging-face-pipeline/blob/master/hugging_face_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this line only on Colab, it installs the transformers package
!pip install transformers

# Hugging Face 🤗
___

We have now learned about APIs and how to use them. In particular, we have seen how to use APIs to obtain data from web sources, but APIs are much more powerful than this. For instance, Google, Microsoft, Amazon, and others let you access some of their deep learning models through an API. While this can be very useful to get your business started quickly, it's generally fairly pricy and can be somewhat difficult to set up. Due to this, we decided to show what you can expect from such APIs by showing you Hugging Face models.

[Hugging Face(https://huggingface.co/) is an AI community which provides many pre-trained deep learning models. From computer vision to sentiment classification, they have it all. In fact, they also provide an API, but it is not free.

The main difference between Hugging Face and some API like Google Vision is that with Hugging Face, **the model runs on your machine**, whereas, with Google Vision, you make use of their servers, i.e., you will need a fairly powerful machine to run Hugging Face models, whereas you could make use of Google Vision on any device, as long as it can connect to their API.

Hugging Face allows to make nice examples in a notebook that you can play with, this would not be doable with any other deep learning API that I know of.

___
## Transformers

Introduced in the seminal paper of [Vaswani et al. (2017)](https://arxiv.org/abs/1706.03762), Transformers are state-of-the-art machine learning models for sequence-based tasks (note: a vision-based task can often be stated as a sequence-based task as well, i.e., there are so called *vision transformers*).

Transformers are typically fairly complex. To simplify greatly, we can think of them as sequence-based models that primarily use [self-attention](https://en.wikipedia.org/wiki/Attention_(machine_learning)) to propagate information along the time dimension. This notebook will not delve into the inner workings of transformers. Instead, it will provide you with a few very simple examples to illustrate how powerful transformers can be on a flurry of tasks.

[Hugging Face](https://huggingface.co/) provides access to pre-trained transformer models, making it easy to run intricate AI models in a few lines of code. 

### Inference with pre-trained models

For now, we will focus on how to use 🤗 Transformers for inference. In deep learning, when we talk about *inference*, we mean *putting the learned capabilities of the model to work*. I.e., training the model does not belong to inference.

Hugging Face provides the `pipeline` function for inference with their pre-trained models. The `pipeline` function can accommodate many different types of tasks, amongst others:

+ Image classification
+ Sentiment analysis
+ Speech recoginition
+ Text generation
+ Text summarization
+ Translation
+ Visual question answering

This notebook only provides some examples of the `pipeline` function with the goal to show how easy it is to use complex AI models. As with the other notebooks, we suggest you play around with the codes and try things out!

In [ ]:
# Import the pipeline function from the transformers module
from transformers import pipeline, set_seed

# Import further modules to display images from URLs
from PIL import Image
import requests
from io import BytesIO
# Regular expressions to clean text
import re

# Helper to read an image from URL
get_image_from_url = lambda url: Image.open(BytesIO(requests.get(url).content))

#### Sentiment analysis

In [ ]:
# Load a pre-trained model for sentiment-analysis (text-classification)
sentiment_classifier = pipeline(
    task="sentiment-analysis", 
    model="siebert/sentiment-roberta-large-english"
)

In [ ]:
# Running the model on a single sentence is as easy as it gets!
sentiment_classifier("I had so much fun in this data science class!")

The sentiment classifier always outputs a list of dictionaries, where the items have a **label** key and a **score** key. The label is either **POSITIVE** or **NEGATIVE** and the score is closer to 1 if the model is *confident about the chosen label* and close to zero otherwise.

The output is a list of dictionaries, because we can also pass a list of sentences to classify!

In [ ]:
# Define some sentences to classify
sentences = [
    "This data science class was fairly difficult.",
    "It was a lot to process.",
    "I didn't know how to feel about the bootcamp before the break, but now I really like data science!",
    "I didn't really understand anything but it was fun.",
    "I could have spent my time better."
]
# Classify the sentences
sentiment_classifier(sentences)

Pretty impressive, don't you think? Even when the sentence structure is very similar, transformers are often able to correctly extract the meaning, see the example below.

In [ ]:
# Define some very 'similar' sequences in structure to see how the classifier does
sentences = [
    "This is not an elegant piece of code, it's a horrible hack.",
    "This is not a horrible hack, it's an elegant piece of code."
]
# Classify the sentences
sentiment_classifier(sentences)

#### Image classification

Of course, classifying text as negative or positive is not the only thing we can do with transformers. What about **image classification**?

In [ ]:
# Use a pre-trained model for image classification
image_classifier = pipeline(
    task="image-classification", 
    model="google/vit-base-patch16-224"
)

In [ ]:
# Replace the URL with any image you would like to classify
img_url = "https://cdn.download.ams.birds.cornell.edu/api/v1/asset/302473191/1800"
get_image_from_url(img_url).resize((200, 150)) # Display the image

In [ ]:
# Classify the image (note that we can directly pass the URL, no need for PIL)
image_classifier(img_url) 

As you can see from the above classification, Google's vision transformer is really confident about this image being a Chickadee, and it is correct! But this was not very difficult, perhaps we can try something more confusing...

In [ ]:
# Replace the URL with any image you would like to classify
img_url = "https://cdn.shopify.com/s/files/1/1832/0821/files/catshark.jpg?v=1649869148"
get_image_from_url(img_url).resize((200, 200)) # Display the image

In [ ]:
# Classify the image (note that we can directly pass the URL, no need for PIL)
image_classifier(img_url) 

Even if our cat is dressed up as a shark, Google's vision transformer is able to recognize it, although it's much less confident in the classification.

#### Visual question answering
Oh, and we can also ask our transformers questions about the image!

In [ ]:
# Use a pre-trained model for visual question answering
question_answerer = pipeline(
    task="vqa",
    model="dandelin/vilt-b32-finetuned-vqa"
)

In [ ]:
# Ask the model a question about our image
question_answerer(image=img_url, question="What is the cat wearing?")

#### Text summarization
Transformers can also be used for text summarization, take the following examples which summarizes some recent news articles.

I have collected three recent articles from [CNN](https://edition.cnn.com/articles) into text files in `articles/`:

+ [`articles/lottery.txt`](https://raw.githubusercontent.com/JLDC/hugging-face-pipeline/master/articles/lottery.txt)
+ [`articles/plane_crash.txt`](https://raw.githubusercontent.com/JLDC/hugging-face-pipeline/master/articles/plane_crash.txt)
+ [`articles/havard_museum.txt`](https://raw.githubusercontent.com/JLDC/hugging-face-pipeline/master/articles/havard_museum.txt)

If you want to assess how well the summarizer is performing, go have a look at the short articles in plaintext.

In [ ]:
# Use a pre-trained model for text summarization
summarizer = pipeline(
    task="summarization",
    model="sshleifer/distilbart-cnn-12-6"
)

In [ ]:
# Change the content of the article variable to access another of the three articles
article = "plane_crash"
# Get the article from GitHub and clean special characters
url = f"https://raw.githubusercontent.com/JLDC/hugging-face-pipeline/master/articles/{article}.txt"
content = re.sub("\s+", " ", requests.get(url).text)
content # Display the content of the article

In [ ]:
# Summarize the above content
summarizer(content)

#### Text generation
Keeping the best for last, let's look at text generation. We can also use transformers to generate text by passing the beginning of a sentence, it can generate random continuations of the initial sentence.

In [ ]:
# Use a pre-trained model for text summarization
generator = pipeline(
    task="text-generation",
    model="gpt2"
)

In [ ]:
set_seed(99) # Comment this out to get different propositions

# Generate 3 continutation of this sentence
results = generator("In this data science class, we", num_return_sequences=3,
                   max_length=50, early_stopping=True)

# Print results
for result in results:
    print(result["generated_text"])
    print("-"*50) # Print a separating line

Pretty impressive! Be careful, however, these pre-trained models often come with a strong bias. This is because they have been trained on existing texts, and, in a sense, they take over the bias that was entailed in those texts. Here are two illustrative examples:

In [ ]:
# Set the seed and generate a text for a male protagonist
set_seed(2)
results = generator("The man is working as a", 
                    max_length=10, num_return_sequences=10)

# Print results
for result in results:
    print(result["generated_text"])
    print("-"*50) # Print a separating line

In [ ]:
# Set the seed and generate a text for a female protagonist
set_seed(2)
results = generator("The woman is working as a", 
                    max_length=10, num_return_sequences=10)

# Print results
for result in results:
    print(result["generated_text"])
    print("-"*50) # Print a separating line